In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("../cache/")


# \n을 기준으로 문장을 잘라버림.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vector_store = FAISS.from_documents(docs, cached_embeddings)

# 1. retriver (모든 document들)
retriver = vector_store.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])
chain = {"context": retriver , "question" :RunnablePassthrough() } | prompt | llm
#retriver는 invoke의 값을 가지고 실행됨.
chain.invoke("victory mansions에 대해서 설명해주세요.")

TypeError: list indices must be integers or slices, not str